In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13916 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13911 ( 0.5%) Users that have less than 1 test interactions


In [4]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [6]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=stacked_train)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

RP3betaRecommender: URM Detected 2 ( 0.0%) users with no interactions.


In [7]:
{'topK': 273, 'shrink': 5.0, 'similarity': 'cosine'}
##ITEMKNNCF Recommender
ItemKNNCF_recommender = ItemKNNCFRecommender(URM_train = stacked_train )
ItemKNNCF_recommender.fit(topK=273, shrink=5.0, similarity="cosine")

ItemKNNCFRecommender: URM Detected 2 ( 0.0%) users with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 1172.73 column/sec. Elapsed time 15.40 sec


In [8]:
similarity_rp3 = RP3Beta_recommender.W_sparse
similarity_itemknn = ItemKNNCF_recommender.W_sparse

In [9]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real,Integer

hyperparameters_range_dictionary = {
    "topK": Integer(10,500),
    "alpha": Real(0.0, 1.0)
}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, similarity_rp3, similarity_itemknn],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 85, 'alpha': 0.11000634266620304}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13911 (100.0%) in 8.81 sec. Users per second: 1579
SearchBayesianSkopt: New best config found. Config 0: {'topK': 85, 'alpha': 0.11000634266620304} - results: PRECISION: 0.1966070, PRECISION_RECALL_MIN_DEN: 0.2008216, RECALL: 0.0566515, MAP: 0.0925833, MAP_MIN_DEN: 0.0946557, MRR: 0.4129092, NDCG: 0.2062200, F1: 0.0879581, HIT_RATE: 0.8529221, ARHR_ALL_HITS: 0.6154456, NOVELTY: 0.0054247, AVERAGE_POPULARITY: 0.6243754, DIVERSITY_MEAN_INTER_LIST: 0.8427978, DIVERSITY_HERFINDAHL: 0.9842737, COVERAGE_ITEM: 0.1722687, COVERAGE_ITEM_CORRECT: 0.0677778, COVERAGE_USER: 0.9947797, COVERAGE_USER_CORRECT: 0.8484697, DIVERSITY_GINI: 0.0088452, SHANNON_ENTROPY: 6.9480682, RATIO_DIVERSITY_HERFINDAHL: 0.9846372, RATIO_DIVERSITY_GINI: 0.0331214, RA

EvaluatorHoldout: Processed 13911 (100.0%) in 11.65 sec. Users per second: 1195
SearchBayesianSkopt: New best config found. Config 4: {'topK': 214, 'alpha': 0.5304427028481765} - results: PRECISION: 0.1990080, PRECISION_RECALL_MIN_DEN: 0.2037757, RECALL: 0.0579053, MAP: 0.0952407, MAP_MIN_DEN: 0.0975334, MRR: 0.4219679, NDCG: 0.2104186, F1: 0.0897082, HIT_RATE: 0.8581698, ARHR_ALL_HITS: 0.6312467, NOVELTY: 0.0053847, AVERAGE_POPULARITY: 0.6499927, DIVERSITY_MEAN_INTER_LIST: 0.7974433, DIVERSITY_HERFINDAHL: 0.9797386, COVERAGE_ITEM: 0.1595880, COVERAGE_ITEM_CORRECT: 0.0626281, COVERAGE_USER: 0.9947797, COVERAGE_USER_CORRECT: 0.8536899, DIVERSITY_GINI: 0.0068876, SHANNON_ENTROPY: 6.4911928, RATIO_DIVERSITY_HERFINDAHL: 0.9801004, RATIO_DIVERSITY_GINI: 0.0257910, RATIO_SHANNON_ENTROPY: 0.5201486, RATIO_AVERAGE_POPULARITY: 3.3179260, RATIO_NOVELTY: 0.0284377, 

EvaluatorHoldout: Processed 13916 (100.0%) in 11.28 sec. Users per second: 1233
SearchBayesianSkopt: Config evaluated with evaluato

EvaluatorHoldout: Processed 13911 (100.0%) in 12.94 sec. Users per second: 1075
SearchBayesianSkopt: Config 10 is suboptimal. Config: {'topK': 475, 'alpha': 0.6468237680924359} - results: PRECISION: 0.1990224, PRECISION_RECALL_MIN_DEN: 0.2038554, RECALL: 0.0580028, MAP: 0.0950506, MAP_MIN_DEN: 0.0975214, MRR: 0.4199855, NDCG: 0.2101772, F1: 0.0898266, HIT_RATE: 0.8576666, ARHR_ALL_HITS: 0.6290150, NOVELTY: 0.0053754, AVERAGE_POPULARITY: 0.6568686, DIVERSITY_MEAN_INTER_LIST: 0.7812713, DIVERSITY_HERFINDAHL: 0.9781215, COVERAGE_ITEM: 0.1578714, COVERAGE_ITEM_CORRECT: 0.0618528, COVERAGE_USER: 0.9947797, COVERAGE_USER_CORRECT: 0.8531894, DIVERSITY_GINI: 0.0065369, SHANNON_ENTROPY: 6.3714177, RATIO_DIVERSITY_HERFINDAHL: 0.9784827, RATIO_DIVERSITY_GINI: 0.0244780, RATIO_SHANNON_ENTROPY: 0.5105508, RATIO_AVERAGE_POPULARITY: 3.3530244, RATIO_NOVELTY: 0.0283887, 

Iteration No: 11 ended. Evaluation done at random point.
Time taken: 14.4560
Function value obtained: -0.0951
Current minimum: -0.0

EvaluatorHoldout: Processed 13911 (100.0%) in 7.99 sec. Users per second: 1740
SearchBayesianSkopt: Config 16 is suboptimal. Config: {'topK': 56, 'alpha': 0.07451452225219292} - results: PRECISION: 0.1944576, PRECISION_RECALL_MIN_DEN: 0.1987648, RECALL: 0.0558774, MAP: 0.0913988, MAP_MIN_DEN: 0.0934039, MRR: 0.4088773, NDCG: 0.2038633, F1: 0.0868100, HIT_RATE: 0.8491122, ARHR_ALL_HITS: 0.6082931, NOVELTY: 0.0054476, AVERAGE_POPULARITY: 0.6071688, DIVERSITY_MEAN_INTER_LIST: 0.8681191, DIVERSITY_HERFINDAHL: 0.9868057, COVERAGE_ITEM: 0.1747051, COVERAGE_ITEM_CORRECT: 0.0697159, COVERAGE_USER: 0.9947797, COVERAGE_USER_CORRECT: 0.8446796, DIVERSITY_GINI: 0.0095557, SHANNON_ENTROPY: 7.1549941, RATIO_DIVERSITY_HERFINDAHL: 0.9871700, RATIO_DIVERSITY_GINI: 0.0357820, RATIO_SHANNON_ENTROPY: 0.5733399, RATIO_AVERAGE_POPULARITY: 3.0993289, RATIO_NOVELTY: 0.0287704, 

Iteration No: 17 ended. Evaluation done at random point.
Time taken: 8.7462
Function value obtained: -0.0914
Current minimum: -0.095

EvaluatorHoldout: Processed 13911 (100.0%) in 12.61 sec. Users per second: 1103
SearchBayesianSkopt: Config 22 is suboptimal. Config: {'topK': 290, 'alpha': 0.42087376470893756} - results: PRECISION: 0.1979009, PRECISION_RECALL_MIN_DEN: 0.2025626, RECALL: 0.0574752, MAP: 0.0944122, MAP_MIN_DEN: 0.0968316, MRR: 0.4181977, NDCG: 0.2089425, F1: 0.0890796, HIT_RATE: 0.8531378, ARHR_ALL_HITS: 0.6254794, NOVELTY: 0.0053738, AVERAGE_POPULARITY: 0.6573011, DIVERSITY_MEAN_INTER_LIST: 0.7817299, DIVERSITY_HERFINDAHL: 0.9781674, COVERAGE_ITEM: 0.1577607, COVERAGE_ITEM_CORRECT: 0.0611330, COVERAGE_USER: 0.9947797, COVERAGE_USER_CORRECT: 0.8486842, DIVERSITY_GINI: 0.0065101, SHANNON_ENTROPY: 6.3650273, RATIO_DIVERSITY_HERFINDAHL: 0.9785285, RATIO_DIVERSITY_GINI: 0.0243776, RATIO_SHANNON_ENTROPY: 0.5100387, RATIO_AVERAGE_POPULARITY: 3.3552323, RATIO_NOVELTY: 0.0283805, 

Iteration No: 23 ended. Evaluation done at random point.
Time taken: 14.2456
Function value obtained: -0.0944
Current minimum: -0.

...

Iteration No: 68 ended. Search finished for the next optimal point.
Time taken: 7.6103
Function value obtained: -0.0973
Current minimum: -0.0979
Iteration No: 69 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 219, 'alpha': 1.0}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13911 (100.0%) in 6.68 sec. Users per second: 2083
SearchBayesianSkopt: Config 68 is suboptimal. Config: {'topK': 219, 'alpha': 1.0} - results: PRECISION: 0.2050536, PRECISION_RECALL_MIN_DEN: 0.2122930, RECALL: 0.0627790, MAP: 0.0973633, MAP_MIN_DEN: 0.1013490, MRR: 0.4262283, NDCG: 0.2166892, F1: 0.0961276, HIT_RATE: 0.8690245, ARHR_ALL_HITS: 0.6413021, NOVELTY: 0.0057006, AVERAGE_POPULARITY: 0.5335706, DIVERSITY_MEAN_INTER_LIST: 0.8509980, DIVERSITY_HERFINDAHL: 0.9850937, COVERAGE_ITEM: 0.1920926, COVERAGE_ITEM_CORRECT: 0.0892076, COVERAGE_USER: 0.9947797, COVERAGE_USER_CORRECT: 0.8644880, DIVERSITY_GI

Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 9.1592
Function value obtained: -0.0920
Current minimum: -0.0979
Iteration No: 76 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 491, 'alpha': 1.0}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13911 (100.0%) in 6.45 sec. Users per second: 2158
SearchBayesianSkopt: Config 75 is suboptimal. Config: {'topK': 491, 'alpha': 1.0} - results: PRECISION: 0.2050536, PRECISION_RECALL_MIN_DEN: 0.2122930, RECALL: 0.0627790, MAP: 0.0973633, MAP_MIN_DEN: 0.1013490, MRR: 0.4262283, NDCG: 0.2166892, F1: 0.0961276, HIT_RATE: 0.8690245, ARHR_ALL_HITS: 0.6413021, NOVELTY: 0.0057006, AVERAGE_POPULARITY: 0.5335706, DIVERSITY_MEAN_INTER_LIST: 0.8509980, DIVERSITY_HERFINDAHL: 0.9850937, COVERAGE_ITEM: 0.1920926, COVERAGE_ITEM_CORRECT: 0.0892076, COVERAGE_USER: 0.9947797, COVERAGE_USER_CORRECT: 0.8644880, DIVERSITY_GI

Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 7.0887
Function value obtained: -0.0799
Current minimum: -0.0979
Iteration No: 83 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 423, 'alpha': 0.9018436762788193}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13911 (100.0%) in 12.48 sec. Users per second: 1115
SearchBayesianSkopt: Config 82 is suboptimal. Config: {'topK': 423, 'alpha': 0.9018436762788193} - results: PRECISION: 0.2015384, PRECISION_RECALL_MIN_DEN: 0.2069091, RECALL: 0.0595669, MAP: 0.0962522, MAP_MIN_DEN: 0.0991963, MRR: 0.4230468, NDCG: 0.2129455, F1: 0.0919554, HIT_RATE: 0.8628424, ARHR_ALL_HITS: 0.6351543, NOVELTY: 0.0053880, AVERAGE_POPULARITY: 0.6529519, DIVERSITY_MEAN_INTER_LIST: 0.7772959, DIVERSITY_HERFINDAHL: 0.9777240, COVERAGE_ITEM: 0.1570962, COVERAGE_ITEM_CORRECT: 0.0640678, COVERAGE_USER: 0.9947797, COVERAGE_USER_C

Iteration No: 89 ended. Search finished for the next optimal point.
Time taken: 10.8255
Function value obtained: -0.0951
Current minimum: -0.0979
Iteration No: 90 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 250, 'alpha': 0.5839713272273188}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13911 (100.0%) in 11.98 sec. Users per second: 1161
SearchBayesianSkopt: Config 89 is suboptimal. Config: {'topK': 250, 'alpha': 0.5839713272273188} - results: PRECISION: 0.1987132, PRECISION_RECALL_MIN_DEN: 0.2034487, RECALL: 0.0578064, MAP: 0.0950690, MAP_MIN_DEN: 0.0975028, MRR: 0.4206442, NDCG: 0.2100748, F1: 0.0895596, HIT_RATE: 0.8534253, ARHR_ALL_HITS: 0.6295510, NOVELTY: 0.0053778, AVERAGE_POPULARITY: 0.6548741, DIVERSITY_MEAN_INTER_LIST: 0.7858437, DIVERSITY_HERFINDAHL: 0.9785787, COVERAGE_ITEM: 0.1583698, COVERAGE_ITEM_CORRECT: 0.0624066, COVERAGE_USER: 0.9947797, COVERAGE_USER_

Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 15.1371
Function value obtained: -0.0967
Current minimum: -0.0979
Iteration No: 97 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 12, 'alpha': 0.9169597759849392}
ItemKNNSimilarityHybridRecommender: URM Detected 2 ( 0.0%) users with no interactions.
EvaluatorHoldout: Processed 13911 (100.0%) in 5.99 sec. Users per second: 2323
SearchBayesianSkopt: Config 96 is suboptimal. Config: {'topK': 12, 'alpha': 0.9169597759849392} - results: PRECISION: 0.1904033, PRECISION_RECALL_MIN_DEN: 0.1951697, RECALL: 0.0560829, MAP: 0.0872988, MAP_MIN_DEN: 0.0897918, MRR: 0.3992484, NDCG: 0.1987853, F1: 0.0866447, HIT_RATE: 0.8483934, ARHR_ALL_HITS: 0.5883986, NOVELTY: 0.0055472, AVERAGE_POPULARITY: 0.5679467, DIVERSITY_MEAN_INTER_LIST: 0.8847709, DIVERSITY_HERFINDAHL: 0.9884707, COVERAGE_ITEM: 0.1791904, COVERAGE_ITEM_CORRECT: 0.0769699, COVERAGE_USER: 0.9947797, COVERAGE_USER_COR